# **Install packages**

In [ ]:
!pip install tensorflow-gpu==2.4.0 tensorboard==2.4.0 tensorboard-plugin-profile==2.4.0

     |████████████████████████████████| 394.7 MB 19 kB/s 
     |████████████████████████████████| 10.6 MB 34.8 MB/s 
     |████████████████████████████████| 1.2 MB 29.7 MB/s 
     |████████████████████████████████| 2.9 MB 32.0 MB/s 
     |████████████████████████████████| 462 kB 44.9 MB/s 
     |████████████████████████████████| 3.8 MB 38.4 MB/s 
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.39.0
    Uninstalling grpcio-1.39.0:
      Successfully uninstalled grpcio-1.39.0
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.6.0
    Uninstalling tensorflow-estimator-2.6.0:
      Successfully uninstalled tensorflow-estimator-2.6.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.6.0
    Uninstalling tensorboard-2.6.0:
      Successfully uninstalled tensorboard-2.6.0
  Attempting uninstall: h5py
    Found existing installation: h5py 3.1.0
    Uninstalling h5py-3.1.0:
      Successf

In [ ]:
import os
os.environ['TF_GPU_THREAD_MODE'] = 'gpu_private'

# **Return GPU**

In [6]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [7]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 1581060044347552681, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 11345264640
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 3793168091230905692
 physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"]

# **Mount the Google Drive**



In [9]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


# **Install packages**

In [10]:
from datetime import datetime

import numpy as np
import tensorflow as tf

from sklearn.utils import shuffle
from sklearn.preprocessing import MinMaxScaler

import itertools
from sklearn import linear_model
from sklearn.model_selection import train_test_split
import pandas as pd

import tensorflow.keras
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical

In [11]:
tensorflow.keras.__version__

'2.6.0'

In [12]:
SEED = 0
np.random.seed(SEED)

In [13]:
RAIN_RATE = 2.0 # adjust rate between the two type, rainy and non-rainy
N_PREDICTOR = 15 #adjust predictor numbers

In [16]:
CLOUD_TRAIN_FOLDER = '/content/gdrive/MyDrive/CloudClassificationGPU/train_10mm/'
CLOUD_FLORENCE_FOLDER = '/content/gdrive/MyDrive/CloudClassificationGPU/florence_10mm/'
CLOUD_TEST_FOLDER = '/content/gdrive/MyDrive/CloudClassificationGPU/test_10mm/'

TIME = 'day'
RAIN_CLOUD_FILENAME = TIME + '_rain_imerg.txt'
NORAIN_CLOUD_FILENAME = TIME + '_norain_imerg.txt'

# **Load the data**

In [17]:
def load_data(folder, ratio):
  rain_cloud_path = folder + RAIN_CLOUD_FILENAME
  norain_cloud_path = folder + NORAIN_CLOUD_FILENAME
  print(rain_cloud_path)

  rain_cloud_array = np.loadtxt(rain_cloud_path, delimiter=' ', dtype=np.float32, skiprows=0)
  norain_cloud_array = np.loadtxt(norain_cloud_path, delimiter=' ', dtype=np.float32, skiprows=0)

  rain_cloud_array = rain_cloud_array[~np.isnan(rain_cloud_array).any(axis=1)]
  norain_cloud_array = norain_cloud_array[~np.isnan(norain_cloud_array).any(axis=1)]

  rain_size = rain_cloud_array.shape[0]
  norain_size = norain_cloud_array.shape[0]

  smaller_size = min(rain_size, norain_size)
  new_rain_size = int(smaller_size // RAIN_RATE)
  new_norain_size = smaller_size

  random_rain_indices = np.random.choice(rain_size, size=new_rain_size, replace=False)
  random_norain_indices = np.random.choice(norain_size, size=new_norain_size, replace=False)
  new_rain_cloud_array = rain_cloud_array[random_rain_indices, :]
  new_norain_cloud_array = norain_cloud_array[random_norain_indices, :]

  cloud_array = np.concatenate((new_rain_cloud_array, new_norain_cloud_array), axis=0)
  cloud_array = shuffle(cloud_array)
  return cloud_array


In [18]:

cloud_train = load_data(CLOUD_TRAIN_FOLDER, ratio=RAIN_RATE)
cloud_florence = load_data(CLOUD_FLORENCE_FOLDER, ratio=RAIN_RATE)

cloud_train_all = np.concatenate((cloud_train, cloud_florence), axis=0)
cloud_train_all = np.repeat(cloud_train_all, repeats=6, axis=0)
cloud_test = load_data(CLOUD_TEST_FOLDER, ratio=RAIN_RATE)

/content/gdrive/MyDrive/CloudClassificationGPU/train_10mm/day_rain_imerg.txt
/content/gdrive/MyDrive/CloudClassificationGPU/florence_10mm/day_rain_imerg.txt
/content/gdrive/MyDrive/CloudClassificationGPU/test_10mm/day_rain_imerg.txt


In [19]:
def split_data(cloud_arr):
  x = cloud_arr[:, :N_PREDICTOR]
  labels = cloud_arr[:, -1]
  y = to_categorical(labels)
  return x, y, labels

x_org, y_org, labels_org = split_data(cloud_train_all)
x_test, y_test, labels_test = split_data(cloud_test)

In [20]:
scaler = MinMaxScaler()

x_org = scaler.fit_transform(x_org)
x_test = scaler.transform(x_test)

In [21]:
x_train, x_val, y_train, y_val = train_test_split(
    x_org, 
    y_org,
    test_size=0.2, 
    random_state=SEED
)

In [22]:
len(x_train)

2206718

In [23]:
INPUT_DIM = N_PREDICTOR

def build_classifier():
  model = Sequential()
  model.add(Dense(units=512, input_dim=INPUT_DIM, kernel_initializer='normal', activation='relu'))
  model.add(Dense(units=256, kernel_initializer='normal', activation='relu'))
  model.add(Dense(units=128, kernel_initializer='normal', activation='tanh'))
  model.add(Dense(units=2, kernel_initializer='normal', activation='softmax'))
  
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

estimator = KerasClassifier(build_fn=build_classifier)

In [24]:
LOGS = '/content/gdrive/MyDrive/CloudClassificationGPU/logs/' + datetime.now().strftime("%Y%m%d-%H%M%S")

tboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir = LOGS,
    histogram_freq = 1,
    profile_batch = (2, 8)
)

EPOCHS = 100

history = estimator.fit(
    x_train,
    y_train,
    epochs=EPOCHS,
    batch_size=128,
    validation_data=(x_val, y_val), 
    verbose=True,
    callbacks = [tboard_callback]
)

Epoch 1/100
17240/17240 [==============================] - 75s 4ms/step - loss: 0.3339 - accuracy: 0.8524 - val_loss: 0.3308 - val_accuracy: 0.8525
Epoch 2/100
17240/17240 [==============================] - 76s 4ms/step - loss: 0.3205 - accuracy: 0.8569 - val_loss: 0.3256 - val_accuracy: 0.8539
Epoch 3/100
17240/17240 [==============================] - 73s 4ms/step - loss: 0.3141 - accuracy: 0.8595 - val_loss: 0.3127 - val_accuracy: 0.8603
Epoch 4/100
17240/17240 [==============================] - 73s 4ms/step - loss: 0.3095 - accuracy: 0.8617 - val_loss: 0.3102 - val_accuracy: 0.8622
Epoch 5/100
17240/17240 [==============================] - 74s 4ms/step - loss: 0.3060 - accuracy: 0.8629 - val_loss: 0.3071 - val_accuracy: 0.8612
Epoch 6/100
17240/17240 [==============================] - 73s 4ms/step - loss: 0.3026 - accuracy: 0.8643 - val_loss: 0.2989 - val_accuracy: 0.8661
Epoch 7/100
17240/17240 [==============================] - 76s 4ms/step - loss: 0.2999 - accuracy: 0.8653 - val_

In [25]:
train_loss = history.history['loss']
val_loss = history.history['val_loss']
train_acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

In [26]:
accuracy_score = estimator.score(x_test, y_test)
print("Accuracy: {0:f}".format(accuracy_score))

3821/3821 [==============================] - 14s 4ms/step - loss: 0.3447 - accuracy: 0.8665
Accuracy: 0.866471


In [27]:
predictions = estimator.predict(x_test)
pred_test = list(predictions)
pred_test = np.array(pred_test)

In [28]:
flag_a = np.logical_and(labels_test == 1, pred_test == 1)
A = labels_test[flag_a]
a = len(A)

flag_b = np.logical_and(labels_test == 0, pred_test == 1)
B = labels_test[flag_b]
b = len(B)

flag_c = np.logical_and(labels_test == 1, pred_test == 0)
C = labels_test[flag_c]
c = len(C)

flag_d = np.logical_and(labels_test == 0, pred_test == 0)
D = labels_test[flag_d]
d = len(D)

In [29]:
PDO = 1.0 * a / (a + c)
POFD = 1.0 * b / (b + d)
FAR = 1.0 * b / (a + b)
Bias = 1.0 * (a + b) / (a + c)
CSI = 1.0 * a / (a + b + c)
AM = 1.0 * (a + d) / (a + b + c + d)
CC = 1.0 * d / (b + d)

In [30]:
print(a, b, c, d)
print(PDO, POFD, FAR, Bias, CSI, AM)

27734 3308 13016 78193
0.6805889570552147 0.040588459037312424 0.10656529862766574 0.7617668711656441 0.6294884016523673 0.8664714399064221


In [31]:
estimator.model.save('my_model')

INFO:tensorflow:Assets written to: my_model/assets
